<a href="https://colab.research.google.com/github/chacha86/pythonai/blob/main/%EC%9B%B9%EC%8A%A4%ED%81%AC%EB%9E%A9%ED%95%912.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

## 브라우저인 척 하기위해 헤더 정보에 브라우저 정보 포함
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

## 해당 url로 요청 보내서 html 문서 받아오기
r = requests.get('https://news.naver.com/main/ranking/popularDay.naver', headers=headers)
    
## html 문서 가져오기
html_doc = r.text

## BeautifulSoup 객체에 넘겨 파싱하기(정보를 태그 단위로 쪼개기)
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')


## css 선택자를 사용하기 위한 함수 =>
### 1. select()  => 결과가 list
### 2. select_one() => 결과가 1개

## 텍스트와 속성값 얻기
### 1. 태그.text
### 2. 태그[key]
###    - key = value

# 1. 언론사 이름과 링크
boxes = soup.select('.rankingnews_box')

journal_list = []

for box in boxes :

  atag = box.select_one('.rankingnews_box_head')
  strong = box.select_one('.rankingnews_name')

  name = strong.text
  link = atag['href']

  journal = {
      'name' : name,
      'link' : link
  }

  journal_list.append(journal)


journal_list


[{'name': '중앙일보',
  'link': 'https://media.naver.com/press/025/ranking?type=popular'},
 {'name': '매일경제',
  'link': 'https://media.naver.com/press/009/ranking?type=popular'},
 {'name': '한국일보',
  'link': 'https://media.naver.com/press/469/ranking?type=popular'},
 {'name': 'JTBC',
  'link': 'https://media.naver.com/press/437/ranking?type=popular'},
 {'name': '조선비즈',
  'link': 'https://media.naver.com/press/366/ranking?type=popular'},
 {'name': '한겨레',
  'link': 'https://media.naver.com/press/028/ranking?type=popular'},
 {'name': '연합뉴스',
  'link': 'https://media.naver.com/press/001/ranking?type=popular'},
 {'name': 'SBS',
  'link': 'https://media.naver.com/press/055/ranking?type=popular'},
 {'name': '이데일리',
  'link': 'https://media.naver.com/press/018/ranking?type=popular'},
 {'name': '한국경제',
  'link': 'https://media.naver.com/press/015/ranking?type=popular'},
 {'name': 'YTN',
  'link': 'https://media.naver.com/press/052/ranking?type=popular'},
 {'name': '헤럴드경제',
  'link': 'https://media.na